In [81]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import zscore
from sklearn.cluster import KMeans

In [ ]:
data_train = pd.read_csv('train.csv')
data_valid = pd.read_csv('valid.csv')

C:\Users\artem\AppData\Local\Temp\ipykernel_14844\2535122673.py:1: DtypeWarning: Columns (51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,131,132,133,135,136,139,140,141,142,143,144,145,146,147,148,149,151,152,155,156,157,158,159,160,161,163,164,165,167,168,169,171,172,173,174,175,176,177,178,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,223,224,227,239,240,241,242,243,244,245,246,459,460,461,467,468,469,515,516,523,524,531,532,539,540,547,548,555,556,563,564,565,567,571,572,573,575,579,580,581,582,583,587,588,595,596,603,604,611,612,613,615,795,796,797,799,1031,1032,1033,1034,1035,1036,1037,1039,1040,1041,1042,1043,1044,1045,1047,1048,1049,1050,1051,1052,1055,1056,1057,1058,1059,1060,1063,1064,1065,1067,1068,1069,1072,1073,1074,1075,1076,1077,1078,1172,1173,1174,1176,1177,1178,1179,1180,1181,1182,1275,1276,1277,1278,1279,1280,1281,1282,1447,1448,1449,145

In [4]:
numeric_columns = data_train.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = data_train.select_dtypes(include=['object']).columns
boolean_columns = data_train.select_dtypes(include=['bool']).columns

print(f"Числовые колонки: {len(numeric_columns)}")
print(f"Категориальные колонки: {len(categorical_columns)}")
print(f"Булевые колонки: {len(boolean_columns)}")


Числовые колонки: 2319
Категориальные колонки: 347
Булевые колонки: 0


In [57]:
# Удаление колонок, где все значения NaN
data_cleaned = data_train.dropna(axis=1, how='all')
data_cleaned = data_cleaned.loc[:, ((data_cleaned == 0) | (data_cleaned.isnull())).mean() < 0.1]
data_cleaned = data_cleaned.loc[:, data_cleaned.nunique() > 10]
data_cleaned = pd.concat([data_cleaned, data_train['target']], axis=1)
data_cleaned = data_cleaned.drop_duplicates()
data_cleaned = data_cleaned[data_cleaned.notna().mean(axis=1) >= 0.5]


# Сравнение до и после удаления
print(f"До удаления: {data_train.shape[1]} колонок")
print(f"После удаления: {data_cleaned.shape[1]} колонок")

До удаления: 2666 колонок
После удаления: 25 колонок


In [63]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14419 entries, 0 to 14455
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   report_date  14419 non-null  object 
 1   client_id    14419 non-null  int64  
 2   col1454      14419 non-null  object 
 3   col2174      14419 non-null  float64
 4   col2182      14419 non-null  float64
 5   col2190      14419 non-null  float64
 6   col2220      14419 non-null  float64
 7   col2221      14419 non-null  float64
 8   col2222      14419 non-null  float64
 9   col2292      14419 non-null  float64
 10  col2293      14419 non-null  float64
 11  col2294      14419 non-null  float64
 12  col2316      14419 non-null  float64
 13  col2317      14419 non-null  float64
 14  col2318      14419 non-null  float64
 15  col2340      14419 non-null  float64
 16  col2341      14419 non-null  float64
 17  col2342      14419 non-null  float64
 18  col2366      14419 non-null  float64
 19  col2388  

In [84]:
# Сначала группируем по client_id, объединяя строки, заполняя пропуски значениями из других строк
merged_data = data_cleaned.groupby('client_id', as_index=False).apply(lambda group: group.ffill().bfill().iloc[0])

# Убедимся, что в результате не осталось дубликатов
merged_data = merged_data.drop_duplicates(subset='client_id')
numeric_df = merged_data.select_dtypes(include=['int64', 'float64'])
numeric_columns_merged = merged_data.select_dtypes(include=[np.number]).columns


C:\Users\artem\AppData\Local\Temp\ipykernel_14844\3323673915.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_data = data_cleaned.groupby('client_id', as_index=False).apply(lambda group: group.ffill().bfill().iloc[0])


In [78]:
merged_data.describe(include='all')

,report_date,client_id,col1454,col2174,col2182,col2190,col2220,col2221,col2222,col2292,...,col2340,col2341,col2342,col2366,col2388,col2389,col2390,col2470,col2663,target
count,4802,4802.000000,4802,4802.000000,4802.000000,4802.000000,4.802000e+03,4.802000e+03,4.802000e+03,4802.000000,...,4.802000e+03,4.802000e+03,4.802000e+03,4802.000000,4802.000000,4802.000000,4802.000000,4644.000000,4802.000000,4802.000000
unique,25,NaN,4802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2023-02-01,NaN,f99a16a8-c0ef-ec11-b822-005056b8a6c4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,515,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,3311.661599,NaN,3.592878,1.292586,1.274677,8.417551e+06,2.222577e+04,2.875413e+06,11.291337,...,8.587924e+06,7.661434e+06,8.119972e+06,1.454573,41.822228,38.041958,39.943558,276.589578,0.259092,0.030404
std,NaN,2123.184025,NaN,2.608011,0.956012,0.975507,4.554948e+06,2.684603e+05,1.388044e+06,6.911579,...,4.515133e+06,4.292370e+06,4.117941e+06,0.666414,15.705274,16.084331,14.685314,285.332438,0.008253,0.171714
min,NaN,1.000000,NaN,1.000000,0.000000,0.000000,1.000000e+03,-1.050000e+04,1.000000e+03,1.000000,...,1.575480e+06,4.196000e+04,1.575480e+06,0.800000,18.820000,1.300000,13.414286,0.000000,0.230229,0.000000
25%,NaN,1502.500000,NaN,3.000000,1.000000,1.000000,5.413778e+06,1.000000e+04,1.968458e+06,6.000000,...,5.439148e+06,4.907621e+06,5.205798e+06,1.000000,30.900000,26.140000,29.450000,89.000000,0.256261,0.000000
50%,NaN,3091.500000,NaN,3.000000,1.000000,1.000000,7.395932e+06,1.000000e+04,2.605140e+06,10.000000,...,7.464088e+06,6.846578e+06,7.126538e+06,1.000000,36.900000,35.510000,36.260000,195.000000,0.256718,0.000000
75%,NaN,4909.000000,NaN,3.000000,1.000000,1.000000,1.042203e+07,1.000000e+04,3.534544e+06,16.000000,...,1.030685e+07,9.430336e+06,9.727697e+06,2.000000,52.670000,49.360000,50.120000,373.250000,0.257803,0.000000


In [83]:
print(f"Число строк до удаления: {data_cleaned.shape[0]}")
print(f"Число строк после удаления: {merged_data.shape[0]}")

Число строк до удаления: 14419
Число строк после удаления: 0


In [72]:
# Пример (должны быть признаки и целевая переменная)
X = numeric_df.drop(columns=['target'])  # Признаки
y = merged_data['target']                # Целевая переменная

# Модель для выбора признаков
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

# Определяем важность признаков
importance = model.feature_importances_
important_features = X.columns[importance > 0.01]  # Выбираем важные

print("Важные признаки:", important_features)

Важные признаки: Index(['client_id', 'col2174', 'col2220', 'col2221', 'col2222', 'col2292',
       'col2293', 'col2294', 'col2316', 'col2317', 'col2318', 'col2340',
       'col2341', 'col2342', 'col2366', 'col2388', 'col2389', 'col2390',
       'col2470', 'col2663'],
      dtype='object')


In [75]:
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(merged_data.select_dtypes(include=[float, int]))

merged_data['cluster'] = clusters
merged_data.head()

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [68]:
data_cleaned.to_csv('cleaned_train.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'cleaned_train.csv'